In [2]:
#########

# from sklearn.metrics.pairwise import cosine_similarity

# def get_recommendation(title, cos_sim=cos_sim):
#     index = indices[title]
#     sim_scores = pd.Series(cos_sim[index])
#     movie_indices = sim_scores.sort_values(ascending=False).head(6).tail(5).index
#     result = df_for_title.title.iloc[movie_indices]
#     return result

#########

In [3]:
# #########

# # 영화이름을 치면 인덱스를 반환
# def get_index(title, df=df_for_title):
#     index = df_for_title.index[(df_for_title['title'] == title)].to_list()
#     return index  

# ######### 

In [4]:
# #########

# # 인덱스를 치면 영화제목 및 시놉시스를 반환
# def get_title(index, df=df_for_title):
#     title = df_for_title['title'][(df_for_title.index == index)], df_for_title['시놉시스'][index]
#     return title

# #########

In [5]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import matplotlib_inline
import missingno as msno
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import Image
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

0. 데이터 준비

In [6]:
import os
# print(os.getcwd())

In [7]:
# 경로설정 버전 1
# 폴더이름 + 파일이름. 
# (노트북의 위치는 중요치 않고. 폴더이름 + 파일이름으로 절대 경로를 생성)

In [8]:
# ver1
base1 = os.path.dirname(r'C:\Users\jlune\carbon\project_JI\0.Model\data\MovieCrawlerFile_disney.csv')
file1 = os.path.basename(r'C:\Users\jlune\carbon\project_JI\0.Model\data\MovieCrawlerFile_disney.csv')
file2 = os.path.basename(r'C:\Users\jlune\carbon\project_JI\0.Model\data\MovieCrawlerFile_netflix.csv')
file3 = os.path.basename(r'C:\Users\jlune\carbon\project_JI\0.Model\data\MovieCrawlerFile_wacha.csv')
base1 +'\\'+file1 # os.path.join(base1,file1)

'C:\\Users\\jlune\\carbon\\project_JI\\0.Model\\data\\MovieCrawlerFile_disney.csv'

In [9]:
# ver2는 노트북이 있는 폴더를 base로 하여. /data 폴더를 만들어 파일들을 넣었다고 가정.
# base dir + 하위 경로
# r'C:\Users\jlune\carbon\project_JI\0.Model + \data\MovieCrawlerFile_disney.csv 형태
os.getcwd() # 현재폴더 경로 확인
pp = os.path.join(os.getcwd()+r'\data\filename')
# pd.read_csv(pp)

In [10]:
temp1 = pd.read_csv(os.path.join(base1, file1))
temp2 = pd.read_csv(os.path.join(base1, file2))
temp3 = pd.read_csv(os.path.join(base1, file3))
print(len(temp1), len(temp2), len(temp3)) # 디즈니 넷플 왓챠

978 3449 5591


In [11]:
978 + 3449 + 5591

10018

In [12]:
temp1.head().columns

Index(['actors', 'director', 'genre', 'imdb_rating', 'just_rating',
       'opening_date', 'posterLink', 'runtime', 'synopsis', 'title_kor'],
      dtype='object')

In [13]:
temp1['Mark'] = 'D' #디즈니
temp2['Mark'] = 'N' #넷플
temp3['Mark'] = 'W' #왓챠
temp1.head(3)

,actors,director,genre,imdb_rating,just_rating,opening_date,posterLink,runtime,synopsis,title_kor,Mark
0,WaltDisney,UbIwerks,애니메이션,80%,80%,1928.0,https://images.justwatch.com/poster/18697230/s592,8min,첫 미키 마우스 애니메이션 개봉작이자 동시 녹음을 활용한 최초의 애니메이션. 배 위...,증기선 윌리,D
1,"ClarenceNash,EstherCampbell,MarionDarlington,P...",BurtGillett,애니메이션,73%,73%,1932.0,https://images.justwatch.com/poster/11993645/s592,8min,A jealous stump threatens two trees that are i...,Flowers and Trees,D
2,"BillyBletcher,PintoColvig,DorothyCompton,MaryM...",BurtGillett,"애니메이션,코미디,가족,음악",74%,74%,1933.0,https://images.justwatch.com/poster/19274730/s592,8min,"아카데미 상을 받은 이 단편에서, 아기 돼지 삼 형제는 각자의 집을 짓는다. 하나는...",아기 돼지 삼형제,D


In [14]:
t1 = pd.concat([temp1, temp2] , axis=0)
t2 = pd.concat([t1,temp3], axis=0)
# t1.info()
t2.info() #개봉일이 opening year로 스트링으로 바꿀것. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10018 entries, 0 to 5590
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   actors        9419 non-null   object 
 1   director      9517 non-null   object 
 2   genre         9773 non-null   object 
 3   imdb_rating   9484 non-null   object 
 4   just_rating   9484 non-null   object 
 5   opening_date  9915 non-null   float64
 6   posterLink    970 non-null    object 
 7   runtime       9915 non-null   object 
 8   synopsis      9896 non-null   object 
 9   title_kor     9915 non-null   object 
 10  Mark          10018 non-null  object 
dtypes: float64(1), object(10)
memory usage: 939.2+ KB


In [15]:
# 컬럼순서및 sorting
#  df = df[['name', 'sex', 'age', 'height']]
# df
y = t2.columns.tolist()

In [16]:
t3 = t2[['title_kor','opening_date','genre','director','actors','synopsis',
         'imdb_rating','just_rating','runtime','posterLink','Mark']]
dt_backborn = t3.copy()  # 백업용 카피
# dt_backborn.to_csv('backup.csv', index=None)

In [17]:
t3.sort_values(by=['opening_date'],ascending=True, inplace=True)
df = t3

### 1. 전처리 및 sorting

In [18]:
# 제목 데이터 확인 및 결측치 분석
# title 결측치가 103개인데 급한대로 수정 못하니 드랍하고 갑니다.
# 추후 데이터 확인시 보완. 할것. 0324
df.isnull().sum()

title_kor        103
opening_date     103
genre            245
director         501
actors           599
synopsis         122
imdb_rating      534
just_rating      534
runtime          103
posterLink      9048
Mark               0
dtype: int64

In [19]:
df = df.dropna(subset=['title_kor'])
len(df)
# len(t3) - len(df) #103개오키

9915

In [20]:
df.isnull().sum() # 시놉의 결측치 19개 드랍
df = df.dropna(subset=['synopsis'])
len(df)


9896

In [21]:
df.isnull().sum() # 장르 데이터 중요하나. 평균값 채울 방법 미고려 . 일단 드랍하고 갑니다.
df = df.dropna(subset=['genre'])

In [22]:
len(df) #9763개로 출발

9763

In [23]:
# 인덱스 리셋. 
df.reset_index(drop=True, inplace=True)

### 2. 

In [24]:
# 텍스트결합용. df 셋팅
# 전략은 row의 data 분리 후 합치기. 제목. 장르 감독 액터 시놉 마크? 
# (마크는 개인화 버전 OTT 별 추천에서나 고려해봐야 할듯)


df.columns

Index(['title_kor', 'opening_date', 'genre', 'director', 'actors', 'synopsis',
       'imdb_rating', 'just_rating', 'runtime', 'posterLink', 'Mark'],
      dtype='object')

In [25]:
df[:1].values

array([['증기선 윌리', 1928.0, '애니메이션', 'UbIwerks', 'WaltDisney',
        '첫 미키 마우스 애니메이션 개봉작이자 동시 녹음을 활용한 최초의 애니메이션. 배 위의 장난꾸러기 선원 미키는 미니를 기쁘게 하기 위해 동물들을 악기 삼아 “짚 속의 칠면조”를 연주한다. 포악한 피트 선장의 화를 돋운 미키는 조리실에서 감자 깎는 일을 하게 된다.',
        '80%', '80%', '8min',
        'https://images.justwatch.com/poster/18697230/s592', 'D']],
      dtype=object)

In [26]:
df[['title_kor','genre']][1:2].values

array([['Flowers and Trees', '애니메이션']], dtype=object)

In [27]:
# 텍스트 결합을 위한 input 준비
df_input = df.iloc[:,0:6] #제목부터 시놉 까지 셀렉
df_input = df_input.drop(['opening_date'], axis=1)

In [28]:
df_input.columns

Index(['title_kor', 'genre', 'director', 'actors', 'synopsis'], dtype='object')

In [29]:
a= df_input[1:2].values
a

array([['Flowers and Trees', '애니메이션', 'BurtGillett',
        'ClarenceNash,EstherCampbell,MarionDarlington,PintoColvig,WaltDisney',
        'A jealous stump threatens two trees that are in love by starting a forest fire. When the rain comes and puts out the fire the forest revives and celebrates the wedding.']],
      dtype=object)

In [30]:
b = a[0][:]
b[:].tolist()
len(b[:].tolist()), b[:].tolist()

(5,
 ['Flowers and Trees',
  '애니메이션',
  'BurtGillett',
  'ClarenceNash,EstherCampbell,MarionDarlington,PintoColvig,WaltDisney',
  'A jealous stump threatens two trees that are in love by starting a forest fire. When the rain comes and puts out the fire the forest revives and celebrates the wedding.'])

In [31]:
intext = " ".join(b[:].tolist())
intext.replace('"','')  #리플레이스 고민. 안먹네.
intext

'Flowers and Trees 애니메이션 BurtGillett ClarenceNash,EstherCampbell,MarionDarlington,PintoColvig,WaltDisney A jealous stump threatens two trees that are in love by starting a forest fire. When the rain comes and puts out the fire the forest revives and celebrates the wedding.'

In [32]:
type(str('1.0'))

str

In [33]:
for i in range(5):
    print(i)

0
1
2
3
4


In [34]:
t_list = []
df_input.fillna(' ', inplace=True) # 후에 공백으로 채우던 결과 변경되는지 확인
for i in range(len(df)):
    s1 = df_input[i:i+1].values
    # print(i)
    try:
        s2 = str(" ".join(s1[0][:].tolist()))
    except:
        print(i)
        pass
    t_list.append(s2)
pd.Series(t_list)     

0       증기선 윌리 애니메이션 UbIwerks WaltDisney 첫 미키 마우스 애니메이...
1       Flowers and Trees 애니메이션 BurtGillett ClarenceNa...
2       아기 돼지 삼형제 애니메이션,코미디,가족,음악 BurtGillett BillyBle...
3       현명한 암탉 애니메이션 WilfredJackson FlorenceGill,Clare...
4       나쁜 늑대 애니메이션 BurtGillett SaraBerner,BillyBletch...
                              ...                        
9758    네 명의 저녁 식사 코미디,로맨스 AlessioMariaFederici Ilenia...
9759    앨리 웡: 돈 웡 코미디 NahnatchkaKhan AliWong 세 번째 넷플릭스...
9760    스루 마이 윈도 드라마,로맨스,코미디 MarçalForés ClaraGalle,Ju...
9761    아빠의 바이올린 드라마 AndaçHaznedaroğlu EnginAltanDüzya...
9762    얼어버린 시간 속에서 역사,드라마 PeterFlinth NikolajCoster-W...
Length: 9763, dtype: object

In [35]:
len(df)

9763

In [36]:
df_U4 = df[['title_kor','synopsis']].rename(columns={"title_kor":"title", "synopsis":'synopsis'})

In [37]:
df_U4['input_t'] = t_list

In [38]:
df_U4

,title,synopsis,input_t
0,증기선 윌리,첫 미키 마우스 애니메이션 개봉작이자 동시 녹음을 활용한 최초의 애니메이션. 배 위...,증기선 윌리 애니메이션 UbIwerks WaltDisney 첫 미키 마우스 애니메이...
1,Flowers and Trees,A jealous stump threatens two trees that are i...,Flowers and Trees 애니메이션 BurtGillett ClarenceNa...
2,아기 돼지 삼형제,"아카데미 상을 받은 이 단편에서, 아기 돼지 삼 형제는 각자의 집을 짓는다. 하나는...","아기 돼지 삼형제 애니메이션,코미디,가족,음악 BurtGillett BillyBle..."
3,현명한 암탉,고전 단편 애니메이션 '현명한 암탉'에 도날드 덕이 데뷔한다. 옥수수를 심고 있는 ...,"현명한 암탉 애니메이션 WilfredJackson FlorenceGill,Clare..."
4,나쁜 늑대,"＇빨간 모자 소녀＇와 늑대, 아기 돼지 삼형제의 이야기를 담은 애니메이션","나쁜 늑대 애니메이션 BurtGillett SaraBerner,BillyBletch..."
...,...,...,...
9758,네 명의 저녁 식사,소울메이트가 부질없다는 로맨틱 코미디가 왔다. 가상의 현실을 배경으로 펼쳐지는 네 ...,"네 명의 저녁 식사 코미디,로맨스 AlessioMariaFederici Ilenia..."
9759,앨리 웡: 돈 웡,세 번째 넷플릭스 스탠드업 스페셜로 돌아온 앨리 웡. 그녀가 자신의 가장 와일드한 ...,앨리 웡: 돈 웡 코미디 NahnatchkaKhan AliWong 세 번째 넷플릭스...
9760,스루 마이 윈도,옆집에 사는 남자를 오랫동안 짝사랑해온 라켈. 가족의 반대에도 불구하고 그 남자 역...,"스루 마이 윈도 드라마,로맨스,코미디 MarçalForés ClaraGalle,Ju..."
9761,아빠의 바이올린,갑작스럽게 아빠를 잃고 고아가 된 소녀. 성공한 바이올린 연주자인 삼촌은 조카를 무...,아빠의 바이올린 드라마 AndaçHaznedaroğlu EnginAltanDüzya...


## 3. 피쳐 벡터라이즈

In [39]:
df_U4.reset_index(drop=True, inplace=True)
sent = []

for i in (range(0,len(df_U4))):
    content = df_U4['synopsis'][i]
    sent.append(content)
len(sent)

9763

In [40]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer()
tfidf_matrix=tfidf.fit_transform(sent)

In [41]:
tfidf_matrix[0].shape
# (1, 160805) 모델 원으 경우. 

(1, 134131)

In [42]:
# cos U
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cos_sim.shape, type(cos_sim)

((9763, 9763), numpy.ndarray)

In [43]:
indices = pd.Series(df_U4.index, index=df_U4.title)
indices.head()

title
증기선 윌리               0
Flowers and Trees    1
아기 돼지 삼형제            2
현명한 암탉               3
나쁜 늑대                4
dtype: int64

In [44]:
# 상위10개
def get_recommendation(title, cos_sim=cos_sim):
    index = indices[title]
    sim_scores = pd.Series(cos_sim[index])
    movie_indices = sim_scores.sort_values(ascending=False).head(11).tail(10).index
    result = df_U4.title.iloc[movie_indices]
    return result

In [45]:
get_recommendation('반지의 제왕: 반지 원정대')

# 9010     반지의 제왕: 두 개의 탑
# 1350      성기사단: 어둠의 마법사
# 7364            마법사의 제자
# 2087       잃어버린 제국을 찾아서
# 2170    신데렐라: 마법 반지의 비밀
# 모델 원. 차이점 생각해볼것 .겹치는 단어가 많이 늘었어. 벡터화에따라 
# 현재는 cos u (가장 좋기도 하다.)에 따라 판단. 단어단위기도 한데. 불용어 처리 및 전처리 후에 테스트 다시 볼 것. 

7980                성기사단: 어둠의 마법사
2706                      마법사의 제자
3071               로덴시아: 마법왕국의 전설
9421                  라야와 마지막 드래곤
2699                     라스트 에어벤더
3802    드래곤 네스트: 평화 기사단 VS 블랙 드래곤
8709                        헤븐퀘스트
8259                         헤드헌터
4201                  우리별 일호와 얼룩소
6434                 스키타이: 불멸의 전사
Name: title, dtype: object

In [46]:
# 연인 중복 키값 에러 확인
get_recommendation('연인')

2985                  엘르
8806    사이다처럼 말이 톡톡 솟아올라
6406             #리얼리티하이
9628              스케이터 걸
9726             더 프리빌리지
515          수잔 브링크의 아리랑
2405     엘 시크레토: 비밀의 눈동자
8556              파리의 인어
9708              히즈 올 댓
3259                늑대소년
Name: title, dtype: object

In [47]:
# get_recommendation('코만도') 중복 키값 에러.  왓챠에서 두개 겹쳐 .

In [48]:
get_recommendation('부산행')
# 2335     웨스트 오브 헬
# 4982       최후의 인류
# 618          최미역행
# 679           러브컷
# 10514    카산드라 크로싱
# Name: title, dtype: object

# 웨스트 오브 헬 만 남고 나머지 순위 왜 바뀌었는지 감독 .배우 등으로 체크. 그후 시각화 고려

6633              웨스트 오브 헬
5139                  7:19
8993            뉴스 오브 더 월드
6177                    달링
9017              프리키 데스데이
9578    이스케이프 룸 2: 노 웨이 아웃
2792                 더티 픽처
6676              워킹 데드 나잇
7173              아메리칸 라이더
5883             홍콩 좀비 히어로
Name: title, dtype: object

In [49]:
def get_index(title, df=df_U4):
    index = df_U4.index[(df_U4['title'] == title)].to_list()
    return index 

get_index('부산행')

[5391]

In [50]:
def get_title(index, df=df_U4):
    title = df_U4['title'][(df_U4.index == index)], df_U4['synopsis'][index]
    return title

get_title(5409)

(5409    핵소 고지
 Name: title, dtype: object,
 '비폭력주의자인 도스는 전쟁으로부터 조국과 소중한 사람들을 지키기 위해 총을 들지 않아도 되는 의무병으로 육군에 자진 입대한다. 총을 들 수 없다는 이유로 필수 훈련 중 하나인 총기 훈련 마저 거부한 도스는 동료 병사들과 군 전체의 비난과 조롱을 받게 된다. 결국 군사재판까지 받게 되지만 끝까지 자신의 신념을 굽히지 않은 도스에게 군 상부는 오키나와 전투에 총기 없이 의무병으로 참전할 것을 허락하는데…')

In [51]:
# 중복검출기 영화이름을 치면. 인덱스 반환
print(df_for_title.index[(df_for_title['title'] == '연인')].to_list())

NameError: name 'df_for_title' is not defined

In [52]:
# 20개만 추려 상관관계 보고자.
alist = []
for i in range(20):
    c = df['title_kor'][i]
    alist.append(c)
# alist
len(alist)
# 베스트셀러, 및 코만도 중복 키 값으로 에러 일어날것. 아래 에러패스 방법

20

In [53]:
alist

['증기선 윌리',
 'Flowers and Trees',
 '아기 돼지 삼형제',
 '현명한 암탉',
 '나쁜 늑대',
 'The Tortoise and the Hare',
 'On Ice',
 'Thru the Mirror',
 "Mickey's Rival",
 'Hawaiian Holiday',
 '외로운 유령들',
 '백설공주와 일곱 난쟁이',
 'Clock Cleaners',
 '황소 페르디난드',
 "Donald's Golf Game",
 'Brave Little Tailor',
 "Mickey's Trailer",
 '환타지아',
 '도날드의 데이트',
 '피노키오']

In [54]:
# 단순 에러 패스. 
df_result = pd.DataFrame()    
for i in alist:
    try : 
        s = pd.DataFrame(get_recommendation(i))
        s.columns = [i]   # 컬럼네임 title => 영화제목으로 변경
        # s.rename(columns={'title':i})
        s.reset_index(drop=True, inplace=True) # 인덱스, 컨캣 시 에러를 띄우는 인덱스 정렬용.
        df_result = pd.concat([df_result, s], axis=1)  # 열 방향으로 컨캣
    except:
        print(i)  # 키 중복으로 겟 레커멘드가. 영화를 특정하지 못함. 인덱스 반환으로 제목 확인. 중복된 제목임.
        pass
# df_result.head()

In [55]:
df_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   증기선 윌리                     10 non-null     object
 1   Flowers and Trees          10 non-null     object
 2   아기 돼지 삼형제                  10 non-null     object
 3   현명한 암탉                     10 non-null     object
 4   나쁜 늑대                      10 non-null     object
 5   The Tortoise and the Hare  10 non-null     object
 6   On Ice                     10 non-null     object
 7   Thru the Mirror            10 non-null     object
 8   Mickey's Rival             10 non-null     object
 9   Hawaiian Holiday           10 non-null     object
 10  외로운 유령들                    10 non-null     object
 11  백설공주와 일곱 난쟁이               10 non-null     object
 12  Clock Cleaners             10 non-null     object
 13  황소 페르디난드                   10 non-null     object
 14  Donald's Golf

In [56]:
# 모든영화 추천 상위 10개씩.
ab_list = df['title_kor'][:]
len(ab_list), ab_list

(9763,
 0                  증기선 윌리
 1       Flowers and Trees
 2               아기 돼지 삼형제
 3                  현명한 암탉
 4                   나쁜 늑대
               ...        
 9758           네 명의 저녁 식사
 9759            앨리 웡: 돈 웡
 9760             스루 마이 윈도
 9761             아빠의 바이올린
 9762          얼어버린 시간 속에서
 Name: title_kor, Length: 9763, dtype: object)

In [57]:
df_result = pd.DataFrame()    
for i in ab_list:
    try : 
        s = pd.DataFrame(get_recommendation(i))
        s.columns = [i]   # 컬럼네임 title => 영화제목으로 변경
        # s.rename(columns={'title':i})
        s.reset_index(drop=True, inplace=True) # 인덱스, 컨캣 시 에러를 띄우는 인덱스 정렬용.
        df_result = pd.concat([df_result, s], axis=1)  # 열 방향으로 컨캣
    except:
        print(i)  # 키 중복으로 겟 레커멘드가. 영화를 특정하지 못함. 인덱스 반환으로 제목 확인. 중복된 제목임.
        pass
df_result.T

덤보
34번가의 기적
신데렐라
이상한 나라의 앨리스
피터 팬
레이디와 트램프
로빈슨 가족
웨스트 사이드 스토리
머나먼 여정
어느 하녀의 일기
정글북
졸업
졸업
혹성탈출
솔라리스
삼총사
데스 위시
소림오조
프리키 프라이데이
장마
인페르노
최후의 카운트다운
캐논볼
디바
애니
기문둔갑
아웃사이더
고스트 버스터즈
코만도
빽 투 더 퓨쳐
코만도
빽 투 더 퓨쳐
탑 건
황진이
탑 건
로보캅
열혈남아
빅
최후의 카운트다운
영원한 사랑
흑사회
드라이빙 미스 데이지
드라이빙 미스 데이지
인어공주
천장지구
아비정전
아비정전
천장지구
적과의 동침
미녀와 야수
알라딘
뉴시즈
흐르는 강물처럼
베토벤
흐르는 강물처럼
동성서취
동방삼협
짱구는 못말려 극장판: 액션가면 VS 하이구레 마왕
순수의 시대
쉰들러 리스트
삼총사
머나먼 여정
쉰들러 리스트
동성서취
짱구는 못말려 극장판: 액션가면 VS 하이구레 마왕
동방삼협
소림오조
쇼생크 탈출
포레스트 검프
작은 아씨들
작은 아씨들
34번가의 기적
남자가 사랑할 때
라이온 킹
블랙 뷰티
쇼생크 탈출
포레스트 검프
온리 유
도망자
비포 선라이즈
나쁜 녀석들
공각기동대
쏘아올린 불꽃, 밑에서 볼까? 옆에서 볼까?
공각기동대
비포 선라이즈
당신이 잠든 사이에
쉘 위 댄스
미션 임파서블
로미오와 줄리엣
쉘 위 댄스
미션 임파서블
접속
자칼
아나콘다
해피 투게더
인생은 아름다워
맨 인 블랙
안나 카레니나
아나콘다
접속
내 남자친구의 결혼식
맨 인 블랙
인생은 아름다워
링
라이언 일병 구하기
트루먼 쇼
택시
라이언 일병 구하기
사랑이 다시 올 때
트루먼 쇼
사랑이 다시 올 때
뮬란
위대한 유산
파이트 클럽
암전
영원한 사랑
집으로 가는 길
매트릭스
애니
소년은 울지 않는다
파이트 클럽
노팅 힐
매트릭스
노팅 힐
메멘토
글래디에이터
화양연화
캐스트 어웨이
와호장룡
왓 위민 원트
패밀리 맨
퍼펙트 스톰
공동경비구역 JSA
브링 잇 온
패밀리 맨
왓 위민 원트
와호장룡
캐스트 어웨이
브링 잇 온
퍼펙트 스톰
글래디에이터
메멘토
공동경비구역 JSA
화양연화
엽기

,0,1,2,3,4,5,6,7,8,9
증기선 윌리,멜로디 타임,터키,신데렐라: 트랩 오브 허,나이트 오브 워킹데드,외로운 유령들,나쁜 늑대,나무 저 너머엔,아리아 : 디 아베니레,출동! 드래곤 구조대 - 송윙 드래곤의 비밀,안녕 알베르토
Flowers and Trees,Fire in the Blood,Lost in White,The Rift,Cobra Mafia,링링,The Shift,Drain The Ocean: WWII,Vizontele,아기 거북 토토의 바다 대모험,Mato Sem Cachorro
아기 돼지 삼형제,Compassion,하나와 앨리스,빤스맨과 네 맘대로 극장,늑대 개,극장판 천원돌파 그렌라간: 홍련편,늑대의 어둠,어카운턴트,어카운턴트,나쁜 늑대,알파 : 위대한 여정
현명한 암탉,나쁜 늑대,3인의 기사,멜로디 타임,미키와 콩나무,경적을 울려요,심슨 가족: 더 무비,유전,유전,체스의 여왕,앵그리 버드 더 무비
나쁜 늑대,멜로디 타임,마지막 버킷리스트,환상의 버섯,거울나라의 앨리스,현명한 암탉,바오,간이역,아기돼지 3형제와 쿵푸랜드,지저귀는 새는 날지 않는다: 돈 스테이 골드,크리스마스에 기적이 찾아온다면
...,...,...,...,...,...,...,...,...,...,...
네 명의 저녁 식사,피의 나무,어쩌다 로맨스,좋은 사람 있으면 소개시켜줘,달링,나의 사랑 그리스,몸을 죽이는 자본의 밥상,내가 사랑했던 모든 남자들에게: P.S. 여전히 널 사랑해,지금 사랑하는 사람과 살고 있습니까?,크리스마스로부터 1000km,XOXO: 엑스오엑스오
앨리 웡: 돈 웡,Monopoly,스탠드업 베스트 2020,앨리 웡: 베이비 코브라,지미 카 - 퍼니 비즈니스,나를 믿어줘: 리사 맥베이 납치 사건,앨리 웡: 성性역은 없다,가브리엘 이글레시아스: 두루두루 쇼,하프 브라더스,쥬라기 펫의 모험,존 멀레이니: 더 컴백 키드
스루 마이 윈도,그토록 오랫동안,얼터 미,베리 굿 걸,이다,쉘부르의 우산,카페 소사이어티,밤에 우리 영혼은,테크노 스와핑,베니스에 눈이 내리면,싱글 올 더 웨이
아빠의 바이올린,윌러비 가족,사운드 오브 뮤직,새벽을 알리는 루의 노래,럭키,홀리 캠프!,범고래 등대,무문관,집 이야기,백설공주: 사라진 아빠를 찾아서,엔드리스


In [62]:
re = df_result.T.copy() 

In [67]:
df_U4.to_csv("input_t.csv")

In [58]:
# 설마, 마사까 다찌마와리와 다찌지지 내가 중복값제거를 안했던가? 드랍 듀플리케이트 ㄷㄷ?
# 중복값이 있다고 할때. 코사인 유사도는 왜 돌았지? 중복된 영화. 시놉이 같나?

# 인플레이스 폴스로.. 듀플리케이트 확인할것. ㅠ

df.nunique()

title_kor       8953
opening_date      88
genre           2469
director        5944
actors          8607
synopsis        9064
imdb_rating     2253
just_rating       80
runtime          198
posterLink       967
Mark               3
dtype: int64

In [59]:
df.drop_duplicates(subset='title_kor', keep='first', inplace=False)
# 엉 위에서 컨캣할때 햇나본데? ㄷㄷㄷ 8953이면.

,title_kor,opening_date,genre,director,actors,synopsis,imdb_rating,just_rating,runtime,posterLink,Mark
0,증기선 윌리,1928.0,애니메이션,UbIwerks,WaltDisney,첫 미키 마우스 애니메이션 개봉작이자 동시 녹음을 활용한 최초의 애니메이션. 배 위...,80%,80%,8min,https://images.justwatch.com/poster/18697230/s592,D
1,Flowers and Trees,1932.0,애니메이션,BurtGillett,"ClarenceNash,EstherCampbell,MarionDarlington,P...",A jealous stump threatens two trees that are i...,73%,73%,8min,https://images.justwatch.com/poster/11993645/s592,D
2,아기 돼지 삼형제,1933.0,"애니메이션,코미디,가족,음악",BurtGillett,"BillyBletcher,PintoColvig,DorothyCompton,MaryM...","아카데미 상을 받은 이 단편에서, 아기 돼지 삼 형제는 각자의 집을 짓는다. 하나는...",74%,74%,8min,https://images.justwatch.com/poster/19274730/s592,D
3,현명한 암탉,1934.0,애니메이션,WilfredJackson,"FlorenceGill,ClarenceNash,PurvPullen",고전 단편 애니메이션 '현명한 암탉'에 도날드 덕이 데뷔한다. 옥수수를 심고 있는 ...,70%,70%,8min,https://images.justwatch.com/poster/241272125/...,D
4,나쁜 늑대,1934.0,애니메이션,BurtGillett,"SaraBerner,BillyBletcher,PintoColvig,DorothyCo...","＇빨간 모자 소녀＇와 늑대, 아기 돼지 삼형제의 이야기를 담은 애니메이션",70%,70%,9min,https://images.justwatch.com/poster/262987309/...,D
...,...,...,...,...,...,...,...,...,...,...,...
9758,네 명의 저녁 식사,2022.0,"코미디,로맨스",AlessioMariaFederici,"IleniaPastorelli,MatildeGioli,GiuseppeMaggio,M...",소울메이트가 부질없다는 로맨틱 코미디가 왔다. 가상의 현실을 배경으로 펼쳐지는 네 ...,6.3 (2k),72%,1시간44분,NaN,N
9759,앨리 웡: 돈 웡,2022.0,코미디,NahnatchkaKhan,AliWong,세 번째 넷플릭스 스탠드업 스페셜로 돌아온 앨리 웡. 그녀가 자신의 가장 와일드한 ...,6.6 (1k),74%,59min,NaN,N
9760,스루 마이 윈도,2022.0,"드라마,로맨스,코미디",MarçalForés,"ClaraGalle,JulioPeña,PilarCastro,HugoArbues,Er...",옆집에 사는 남자를 오랫동안 짝사랑해온 라켈. 가족의 반대에도 불구하고 그 남자 역...,5.5 (6k),76%,1시간52분,NaN,N
9761,아빠의 바이올린,2022.0,드라마,AndaçHaznedaroğlu,"EnginAltanDüzyatan,BelçimBilgin,SelimErdoğan,A...",갑작스럽게 아빠를 잃고 고아가 된 소녀. 성공한 바이올린 연주자인 삼촌은 조카를 무...,6.5 (2k),81%,1시간52분,NaN,N


In [60]:
df_input

,title_kor,genre,director,actors,synopsis
0,증기선 윌리,애니메이션,UbIwerks,WaltDisney,첫 미키 마우스 애니메이션 개봉작이자 동시 녹음을 활용한 최초의 애니메이션. 배 위...
1,Flowers and Trees,애니메이션,BurtGillett,"ClarenceNash,EstherCampbell,MarionDarlington,P...",A jealous stump threatens two trees that are i...
2,아기 돼지 삼형제,"애니메이션,코미디,가족,음악",BurtGillett,"BillyBletcher,PintoColvig,DorothyCompton,MaryM...","아카데미 상을 받은 이 단편에서, 아기 돼지 삼 형제는 각자의 집을 짓는다. 하나는..."
3,현명한 암탉,애니메이션,WilfredJackson,"FlorenceGill,ClarenceNash,PurvPullen",고전 단편 애니메이션 '현명한 암탉'에 도날드 덕이 데뷔한다. 옥수수를 심고 있는 ...
4,나쁜 늑대,애니메이션,BurtGillett,"SaraBerner,BillyBletcher,PintoColvig,DorothyCo...","＇빨간 모자 소녀＇와 늑대, 아기 돼지 삼형제의 이야기를 담은 애니메이션"
...,...,...,...,...,...
9758,네 명의 저녁 식사,"코미디,로맨스",AlessioMariaFederici,"IleniaPastorelli,MatildeGioli,GiuseppeMaggio,M...",소울메이트가 부질없다는 로맨틱 코미디가 왔다. 가상의 현실을 배경으로 펼쳐지는 네 ...
9759,앨리 웡: 돈 웡,코미디,NahnatchkaKhan,AliWong,세 번째 넷플릭스 스탠드업 스페셜로 돌아온 앨리 웡. 그녀가 자신의 가장 와일드한 ...
9760,스루 마이 윈도,"드라마,로맨스,코미디",MarçalForés,"ClaraGalle,JulioPeña,PilarCastro,HugoArbues,Er...",옆집에 사는 남자를 오랫동안 짝사랑해온 라켈. 가족의 반대에도 불구하고 그 남자 역...
9761,아빠의 바이올린,드라마,AndaçHaznedaroğlu,"EnginAltanDüzyatan,BelçimBilgin,SelimErdoğan,A...",갑작스럽게 아빠를 잃고 고아가 된 소녀. 성공한 바이올린 연주자인 삼촌은 조카를 무...


XGboost 그러나 타겟값이 없는데. 

In [ ]:
예로서 액션범죄스릴러 가 아닌 나머지로 할까?